In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import flopy
import lumpyrem as lr
from lumpyrem import run
from lumpyrem.lr2series import TimeSeries
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
bins_pth = os.path.join('..', 'bins', 'win') if 'nt' in os.name else os.path.join('..', 'bins', 'linux') # Binaries

In [ ]:
lr_ws = os.path.join('..', 'temp_lr_prep') # Lumprem model workspace.  Safe to delete.

In [ ]:
lumprem_input_pth = os.path.join('..', 'data', 'lumprem_input') # Input data curated in a previous notebook
os.listdir(lumprem_input_pth)

In [ ]:
if not os.path.exists(lr_ws):
    os.mkdir(lr_ws)

In [ ]:
for file in os.listdir(lumprem_input_pth):
    shutil.copyfile(lumprem_input_pth+'\\'+file, lr_ws+'\\'+file) #Works for Windows. Does it work for Linux?

In [ ]:
os.listdir(lr_ws)

## Setup recharge model
What about EVT?

In [ ]:
# initiate the model and write the in files. What about rbuf & mbuf?
# landuse model for recharge, evap
lr_lu1 = lr.lumprem.Model(
    model_name = 'lu1',
    workspace = lr_ws,
    irrigvolfrac = 0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
    maxvol = 0.2, # Varför valde jag 0.2 istället för 0.5 som är default?
    vol = 0.1,
    ks = 43.2 # 43.2 m/day = 0.0005 m/s ≈ reasonable initial guess considering local geology
)

In [ ]:
lr_lu1.write_model(
    start_date = '01/01/2016',
    end_date = '01/01/2020',
    noutdays='monthly',
    mxiter = 500,
)
lr_lu1.run_model(version=2)

In [ ]:
# get model results
results = lr_lu1.get_results()
display(results.head())

In [ ]:
display(results.tail())

In [ ]:
fig = plt.figure(figsize=(10, 6), dpi=50)
ax = fig.add_subplot(1, 1, 1)
plt.ticklabel_format(axis='both', style='plain', useOffset=False) #Show coordinates
ax.set_title('lr_lu1 model', fontsize=18)

results.plot('days','net_recharge', ax=ax)
results.plot('days', 'gw_pot_evap', ax=ax)

plt.legend()
plt.show()

## Setup GHB models

GHB Models:

In [ ]:
initial_ghb_bheads = {
    'red': 142.5,
    'orange': 142.0,
    'yellow': 141.,
    'limegreen': 139.5,
    'royalblue': 135.,
    'blueviolet': 137.5,
    'magenta': 139.5,
}

In [ ]:
lr_ghb_red = lr.lumprem.Model(
    model_name = 'red',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['red'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_orange = lr.lumprem.Model(
    model_name = 'orange',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['orange'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_yellow = lr.lumprem.Model(
    model_name = 'yellow',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['yellow'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_limegreen = lr.lumprem.Model(
    model_name = 'limegreen',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['limegreen'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_royalblue = lr.lumprem.Model(
    model_name = 'royalblue',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['royalblue'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_blueviolet = lr.lumprem.Model(
    model_name = 'blueviolet',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['blueviolet'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
lr_ghb_magenta = lr.lumprem.Model(
    model_name = 'magenta',
    workspace = lr_ws,
    factor1 = 30,
    factor2 = 20,
    power = 2,
    offset = initial_ghb_bheads['magenta'],
    maxvol = 0.2,
    vol = 0.03,
    surface = 200,
    ks = 43.2,
    irrigvolfrac=0.0,
    rainfile = 'rainfall.dat',
    epotfile = 'evapot.dat',
)

In [ ]:
ghb_models = [lr_ghb_red, lr_ghb_orange, lr_ghb_yellow, lr_ghb_limegreen, lr_ghb_royalblue, lr_ghb_blueviolet, lr_ghb_magenta]
for model in ghb_models:
    model.write_model(
        start_date = '01/01/2016',
        end_date = '01/01/2020',
        noutdays='monthly',
        mxiter = 500,
    )
    model.run_model(version=2)

In [ ]:
lr_ghb_red_df = lr_ghb_red.get_results()

In [ ]:
# plot the model results
import seaborn as sb
sb.lineplot(data=lr_ghb_red_df, x='days',y='elevation', hue='lumprem_model_name', alpha=0.5);

# load the simulation

In [ ]:
org_model_ws = os.path.join('..', 'base_model_files')
os.listdir(org_model_ws)

In [ ]:
tmp_model_ws = os.path.join('..', 'temp_flopy_lumprem') # Safe to delete
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
os.listdir(tmp_model_ws)

In [ ]:
ml_name = 'hagfors_1'

In [ ]:
sim = flopy.mf6.MFSimulation.load(ml_name, 'mf6', os.path.join(bins_pth, 'mf6'), tmp_model_ws)

In [ ]:
sim.run_simulation()

In [ ]:
gwf = sim.get_model(ml_name)

In [ ]:
disv = gwf.get_package('disv')
rch = gwf.get_package('rch')
ghb = gwf.get_package('ghb')

## Prepare recharge (RCH) timeseries input

In [ ]:
rch_ts = lr.lr2series.TimeSeries(
    ts_file='rch.ts',
    workspace=lr_ws,
    lr_models=[lr_lu1],
    ts_names=['rch'],
    lumprem_output_cols=['net_recharge'],
    methods=['linearend'],
    div_delta_t=True
)


In [ ]:
rch_ts.write_ts()

Select non-ghb cells (cells can't both be rch and ghb):

In [ ]:
#ghb_cell_ids cells in ghb cant be part of rch (will cause fortran error)
ghb_cell_ids = [i[0][1] for i in ghb.stress_period_data.data[0]]

Create rch TS for all non-ghb cells:

In [ ]:
rchcells = np.array(list(range(disv.ncpl.data)), dtype=int)
rchcells[ghb_cell_ids] = -1
rch_spd = [
    ((0, rchcells[i]), 'rch_lu1', 1, 'rch') for i in range(disv.ncpl.data) if rchcells[i] > 0
]
rch_spd = {0: rch_spd, 1: rch_spd}

Display the data-structure of the old recharge package:

In [ ]:
rch.stress_period_data.data

Let's take a more detailed look at one of the stress periods:

In [ ]:
rch.stress_period_data.data[0][:10]
# Current data structure is cellid, recharge

Remove old recharge package:

In [ ]:
gwf.remove_package('rch')

Instantiate new recharge package:

In [ ]:
rch = flopy.mf6.ModflowGwfrch(
    gwf,
    filename='{}.rch'.format(ml_name), 
    pname='rch',
    fixed_cell=True,
    auxiliary='MULTIPLIER',
    auxmultname='MULTIPLIER',
    print_input=True,
    print_flows=True, 
    save_flows=True,
    boundnames=True,
    stress_period_data=rch_spd
)

Display the data-structure of the **new** recharge package, using TS-file and multiplier:

In [ ]:
rch.stress_period_data.data[0][:10]
# Current data structure is cellid, recharge

Initialize TS-file:

In [ ]:
# Initialize ts-file
rch.ts.initialize(filename='rch.ts')

## Prepare general head boundary (GHB) timeseries input

Extend lumpyrem TimeSeries class (ugly hack but gets the job done):

In [ ]:
class GHB_TS(TimeSeries):
    def write_ts(self):
        """Writes the MODFLOW6 timeseries file.

        Parameters
        ----------
        """
        #number of columns to include in the ts file
        count = len(self.ts_names)
        ts_file = os.path.join(self.workspace, self.ts_file+'.in')

        with open(ts_file, 'w') as f:
            for model in self.lr_models:
                model_name = model.lumprem_model_name
                f.write('READ_LUMPREM_OUTPUT_FILE lr_'+model_name+'.out '+str(count)+'\n')
                f.write('#  my_name     LUMPREM_name      divide_by_delta_t?\n\n')

                for col in range(count):
                    f.write("\t{0}\t\t{1}\t\t{2}".format('ghb_'+model_name, self.lumprem_output_cols[col],self.div_delta[col]+'\n'))
                f.write('\n\n')

            f.write('WRITE_MF6_TIME_SERIES_FILE '+self.ts_file+' '+str(count*len(self.lr_models))+' '+str(self.timeoffset)+'\n')
            f.write("#\t{0}\t\t{1}\t\t{2}\t\t{3}\t\t{4}".format('ts_name','scale','offset','mf6method','time_offset_method\n\n'))
            for model in self.lr_models:
                model_name = model.lumprem_model_name
                for col in range(count):
                        f.write("\t{0}\t\t{1}\t\t{2}\t\t{3}\t{4}\t{5}".format('ghb_'+model_name, self.scales[col],self.offsets[col],self.methods[col], self.time_offset_method[col], '#'+model_name+'\n'))

        f.close()
        print('MF6 timeseries file '+ts_file+' written to:\n'+ts_file)
        
        #write ts file
        filename = self.ts_file
        path = self.workspace
        run.run_process('lr2series', commands=[filename+'.in'],path=path)

Write TS-file:

In [ ]:
ghb_ts = GHB_TS(
    ts_file='ghb.ts',
    workspace=lr_ws,
    lr_models=ghb_models,
    ts_names=['ghb'],
    lumprem_output_cols=['elevation'],
    methods=['linear'],
    div_delta_t=False
)


In [ ]:
ghb_ts.write_ts()

Make elevation/bhead a multiplier of the TS-data...?????!?!?!?!?

Display the data-structure of the old GHB package:

In [ ]:
# Current data structure is cellid, bhead, cond, boundname
ghb.stress_period_data.data

Construct new GHB stress period data:

In [ ]:
bhead = ['ghb_'+i for i in ghb.stress_period_data.data[0].boundname]
cond = [86.4 for i in ghb.stress_period_data.data[0].boundname]
bname = [i for i in ghb.stress_period_data.data[0].boundname]

In [ ]:
# Kanske behöver vara tuple
ghb_new_data = [[(0, cell_id), bhead, cond, bname] for cell_id, bhead, cond, bname in zip(ghb_cell_ids, bhead, cond, bname)]

In [ ]:
ghb_new_spd = {0: ghb_new_data}

In [ ]:
ghb_new_spd

Remove old GHB package:

In [ ]:
gwf.remove_package('ghb')

Instantiate new GHB package:

In [ ]:
ghb = flopy.mf6.modflow.mfgwfghb.ModflowGwfghb(
    gwf,
    print_input=True,
    print_flows=True,
    save_flows=True,
    boundnames=True, 
    pname='ghb',
    stress_period_data=ghb_new_spd
)

Display the new data structure:

In [ ]:
ghb.stress_period_data.data

In [ ]:
# Initialize ts-file
ghb.ts.initialize(filename='ghb.ts')

Write simulation files:

In [ ]:
sim.write_simulation()

Move TS files:

In [ ]:
ts_files = ['rch.ts', 'ghb.ts']
for file in ts_files:
    lumprem_ts = os.path.join(lr_ws, file)
    flopy_ts = os.path.join(tmp_model_ws, file)
    shutil.move(lumprem_ts, flopy_ts)

Move template files:

In [ ]:
template_files = [i for i in os.listdir(lr_ws) if '.tpl' in i]

In [ ]:
for file in template_files:
    lumprem_template = os.path.join(lr_ws, file)
    flopy_template = os.path.join(tmp_model_ws, file)
    shutil.move(lumprem_template, flopy_template)

In [ ]:
sim.run_simulation()

In [ ]:
# check the output
csv = os.path.join(tmp_model_ws,'head.obs.csv')
df = pd.read_csv(csv)

In [ ]:
df.columns

In [ ]:
df['B10-2'].plot()

In [ ]:
headfile = '{}.hds'.format(ml_name)
fname = os.path.join(tmp_model_ws, headfile)
hds = flopy.utils.binaryfile.HeadFile(fname)
h = hds.get_data()

In [ ]:
fig = plt.figure(figsize=(18, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_title("Cross section plot")

xsection = np.array([(427000, 6654690), (427540, 6655230)])
xsect = flopy.plot.PlotCrossSection(model=gwf, line={"line": xsection})
patch_collection = xsect.plot_array(h, head=h, alpha=0.5)
line_collection = xsect.plot_grid()
cb = plt.colorbar(patch_collection, shrink=0.75)

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=100)

axs = fig.subplot_mosaic([['Left', 'TopRight'],['Left', 'BottomRight']],
                          gridspec_kw={'width_ratios':[2, 1]})

axs['Left'].set_title('Layer 1')
axs['TopRight'].set_title('Layer 2')
axs['BottomRight'].set_title('Layer 3')

for index, ax in enumerate(axs):
    modelmap = flopy.plot.PlotMapView(model=gwf, ax=axs[ax])
    linecollection = modelmap.plot_grid(lw=0.25)
    contours = modelmap.contour_array(h[index])
    
plt.tight_layout()
plt.show()

In [ ]:
#Check the budget
mf_list = flopy.utils.Mf6ListBudget(os.path.join(tmp_model_ws, f"{gwf.name}.lst"), timeunit='days') #MF6ListBudget is different from MfListBudget...*sigh*
incremental, cumulative = mf_list.get_budget()
incrementaldf, cumulativedf = mf_list.get_dataframes(start_datetime="01-09-2017")

incrementaldf

Delete LR to start over:

In [ ]:
shutil.rmtree(lr_ws)